## Shortest Path (SP) Problem

In [1]:
import numpy as np
import pandas as pd
import os
import igraph as ig
import networkx as nx
import random
from gurobipy import Model,GRB,LinExpr,quicksum
import cartopy
import cartopy.crs as ccrs
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.axes._axes import _log as matplotlib_axes_logger
matplotlib_axes_logger.setLevel('ERROR')
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.patheffects as PathEffects
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
from pyomo.environ import *

V = {'TX':1,
     'AK':2,
     'LA':3,
     'TN':4,
     'AL':5,
     'NC':6}

E = {('TX','AK'):446,
     ('TX','LA'):432,
     ('AK','LA'):343,
     ('AK','TN'):347,
     ('AK','AL'):463,
     ('LA','TN'):587,
     ('LA','AL'):365,
     ('TN','NC'):539,
     ('AL','TN'):281,
     ('AL','NC'):569}

In [2]:
model = ConcreteModel()

# Define sets
model.vertices = Set(initialize=V.keys())
model.edges    = Set(initialize=E.keys())

s              = 'TX'
t              = 'NC'

# Define parameters
model.distance = Param(model.edges, initialize=E, within=NonNegativeReals)

# Define the decision variables
model.x = Var(model.edges, within=NonNegativeReals)

# Define the objective function
model.obj = Objective(expr=sum(model.distance[(u, v)] * model.x[(u, v)] for (u, v) in model.edges), sense=minimize)

# Define the constraints
model.xlow_balance = ConstraintList()
for v in model.vertices:
    if v == s:
        model.xlow_balance.add(sum(model.x[(v, w)] for w in model.vertices if (v, w) in model.edges)
                               - sum(model.x[(w, v)] for w in model.vertices if (w, v) in model.edges) 
                               == 1)
    elif v == t:
        model.xlow_balance.add(sum(model.x[(v, w)] for w in model.vertices if (v, w) in model.edges)
                               - sum(model.x[(w, v)] for w in model.vertices if (w, v) in model.edges) 
                               == -1)
    else:
        model.xlow_balance.add(sum(model.x[(v, w)] for w in model.vertices if (v, w) in model.edges)
                               - sum(model.x[(w, v)] for w in model.vertices if (w, v) in model.edges) 
                               == 0)
            
# Initialize the variables
for (v, w) in model.edges:
    model.x[(v, w)].value = 0

In [3]:
# Solve the problem
solver = SolverFactory('gurobi')
solver.solve(model)

# Print the results
print('Shortest path:', model.obj())
for (v, w) in model.edges:
    print(f'Going from {v} to {w}: {model.x[(v, w)].value}')

Shortest path: 1332.0
Going from TX to AK: 1.0
Going from TX to LA: 0.0
Going from AK to LA: 0.0
Going from AK to TN: 1.0
Going from AK to AL: 0.0
Going from LA to TN: 0.0
Going from LA to AL: 0.0
Going from TN to NC: 1.0
Going from AL to TN: 0.0
Going from AL to NC: 0.0
